In [ ]:
import os
import re
import numpy as np
import pandas as pd
from collections import defaultdict

# ----- get server data ------
base_dir = "."
# Config mapping
config_map = {
    "eps0.05": "ε = 0.05",
    "eps1": "ε = 1",
    "eps3": "ε = 3",
    "eps8": "ε = 8"
}

server_metrics = ["avg_cpu", "peak_cpu", "avg_mem", "peak_mem"]
config_round_data = {label: defaultdict(list) for label in config_map.values()}

# Parse each config
for config_suffix, label in config_map.items():
    for folder in os.listdir(base_dir):
        if folder.startswith(f"exp_dp_{config_suffix}_r"):
            try:
                full_path = os.path.join(base_dir, folder)
                server_log_dir = [f for f in os.listdir(full_path) if f.startswith("log_server_")][0]
                file_path = os.path.join(full_path, server_log_dir, "resource_usage.txt")

                with open(file_path, "r") as f:
                    current_round = None
                    round_data = {}

                    for line in f:
                        round_match = re.match(r"Round (\d+):", line)
                        if round_match:
                            if current_round is not None:
                                config_round_data[label][current_round].append(round_data)
                            current_round = int(round_match.group(1))
                            round_data = {}
                        else:
                            metric_match = re.match(r"\s*(\w+):\s*([\d.]+)", line)
                            if metric_match:
                                key, val = metric_match.groups()
                                if key in server_metrics:
                                    round_data[key] = float(val)

                    if current_round is not None:
                        config_round_data[label][current_round].append(round_data)

            except Exception as e:
                print(f" Failed to parse {folder}: {e}")



In [ ]:
#Create summary tables 
summary_rows = []
for label, rounds in config_round_data.items():
    all_stats = {m: [] for m in server_metrics}

    for round_dicts in rounds.values():
        for entry in round_dicts:
            for m in server_metrics:
                if m in entry:
                    all_stats[m].append(entry[m])

    summary_rows.append({
        "Epsilon": label,
        "Avg CPU (%)": np.mean(all_stats["avg_cpu"]),
        "Peak CPU (%)": np.mean(all_stats["peak_cpu"]),
        "Avg Memory (MB)": np.mean(all_stats["avg_mem"]),
        "Peak Memory (MB)": np.mean(all_stats["peak_mem"]),
    })

dp_ru_df = pd.DataFrame(summary_rows).set_index("Epsilon")
dp_ru_df


,Avg CPU (%),Peak CPU (%),Avg Memory (MB),Peak Memory (MB)
Epsilon,,,,
ε = 0.05,3.848360,43.070560,1356.195880,1455.786920
ε = 1,3.430495,40.816154,1410.386868,1513.855714
ε = 3,2.330153,28.811374,1423.726641,1545.382366
ε = 8,3.344504,42.307481,1421.506336,1540.807252


In [ ]:
import os
import pandas as pd
from collections import defaultdict

# ----- get client data ------
base_dir = "."
eps_map = {
    "eps0.05": "ε = 0.05",
    "eps1": "ε = 1",
    "eps3": "ε = 3",
    "eps8": "ε = 8"
}

client_summary = []


for cfg_key, label in eps_map.items():
    cpu_vals = []
    mem_vals = []

    for run_folder in os.listdir(base_dir):
        if run_folder.startswith(f"exp_dp_{cfg_key}_r"):
            try:
                run_path = os.path.join(base_dir, run_folder)
                client_dir_name = [f for f in os.listdir(run_path) if f.startswith("log_clients_")][0]
                client_log_path = os.path.join(run_path, client_dir_name, "client_0", "metrics_log.csv")

                if not os.path.exists(client_log_path):
                    continue

                df = pd.read_csv(client_log_path)
                cpu_vals.append((df["Avg CPU (%)"].mean(), df["Peak CPU (%)"].mean()))
                mem_vals.append((df["Avg Memory (MB)"].mean(), df["Peak Memory (MB)"].mean()))

            except Exception as e:
                print(f"Error in {run_folder}: {e}")

    if cpu_vals and mem_vals:
        avg_cpu = sum(x[0] for x in cpu_vals) / len(cpu_vals)
        peak_cpu = sum(x[1] for x in cpu_vals) / len(cpu_vals)
        avg_mem = sum(x[0] for x in mem_vals) / len(mem_vals)
        peak_mem = sum(x[1] for x in mem_vals) / len(mem_vals)

        client_summary.append({
            "Epsilon": label,
            "Avg CPU (%)": round(avg_cpu, 2),
            "Peak CPU (%)": round(peak_cpu, 2),
            "Avg Memory (MB)": round(avg_mem, 2),
            "Peak Memory (MB)": round(peak_mem, 2)
        })


dp_eps_client_df = pd.DataFrame(client_summary).set_index("Epsilon")
dp_eps_client_df


,Avg CPU (%),Peak CPU (%),Avg Memory (MB),Peak Memory (MB)
Epsilon,,,,
ε = 0.05,39.26,49.02,1367.17,1373.32
ε = 1,39.28,49.30,1415.44,1422.06
ε = 3,41.75,48.85,1431.77,1438.20
ε = 8,39.62,49.53,1431.99,1439.80
